<img src="../../data/images/gempy_logo.png" />

# <center> From Maps to Models - Tutorials for structural geological modeling using GemPy and GemGIS</center>

# Model 1 - Horizontal Layers


This first notebook illustrates how to create a simple sample model of horizontal layers in `GemPy`. The model consists of four parallel layers plus basement layers and has an extent of 1000 m by 1000 m with a vertical extent of 600 m. No folded, faulted or truncated layers are present in this model.

If you have not gone through the introduction notebook for the course, please check it out: [Introduction Notebook](../00_introduction_to_structural_modeling.ipynb) ([notebook on Github](https://nbviewer.org/github/cgre-aachen/gemgis_data/blob/main/notebooks/00_introduction_to_structural_modeling.ipynb))


<div class="alert alert-block alert-success">
<b>In this tutorial, you will learn the following:</b> <br>
- How to import input data into <b>GemPy</b> via CSV-Files (comma-separated-values) and what the files have to look like<br>
- How to build a simple model consisting of <b>horizontal layers</b> belonging to <b>one Series</b><br>
- How to visualize the resulting model with cross sections in 2D and the entire model in 3D

</div>

## Contents

1. [Installing GemPy](#installing-gempy)
2. [Importing Libraries](#importing-libraries)
3. [Data Preparation](#data-preparation)
    1. [Importing Interface Points](#importing-interface-points)
    2. [Importing Orientations](#importing-orientations)
4. [GemPy Model Calculation](#gempy-model-calculation)
    1. [Creating the GemPy Model](#creating-the-gempy-model)
    2. [Inspecting the Surfaces](#inspecting-the-surfaces)
    3. [Inspecting the Input Data](#inspecting-the-input-data)
    4. [Map Stack to Surfaces](#map-stack-to-surfaces)
    5. [Plotting Input Data in 2D](#plotting-the-input-data-in-2d)
    6. [Plotting Input Data in 3D](#plotting-the-input-data-in-3d)
    7. [Setting the Interpolator](#setting-the-interpolator)
    8. [Computing the Model](#computing-the-model)
5. [Model Visualization and Post-Processing](#model-visualization-and-post-processing)
    1. [Visualizing Cross Sections of the Computed Model](#visualizing-cross-sections-of-the-computed-model)
    2. [Visualizing the computed model in 3D](#visualizing-the-computed-model-in-3d)
6. [Conclusions](#conclusions)
7. [Outlook](#outlook)
8. [Licensing](#licensing)



<img src="../images/model1.png" width=500/>


The input data is provided as already prepared CSV-files (comma-separated-values) and will be loaded as Pandas `DataFrames` (see Figure below, [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)). The first file contains the interface points, the boundaries between the four stratigraphic units. It should consist of four columns with the headers `X`, `Y`, `Z` for the location of the interface points in meter and `formation` for name of the layer the interface belongs to (see Figure below). The second file contains the orientation of the layers with the same columns as before plus three more columns, `dip`, `azimuth`, and `polarity` indicating the dip the layer and the dip direction (azimuth, see Figure below) in degrees. The dip varies from 0° for horizontal layers to 90° for vertical layers. The azimuth varies from 0° (N) via 180° (S) to 360° (N). Here, we only provide the orientations for one layer. This will be explained later on in more detail. The `polarity` value is mostly set to 1.

<div class="alert alert-block alert-warning">
<b>Important:</b> Interface points always represent the <b>base</b> of the respective layer. However, orientations of layers must not be located at the location of the layer boundaries but may also be located within the layer itself as the orientations will "only" constrain the gradient of the scalar field interpolated by GemPy.   
    
</div>

<img src="../../data/images/model1_fig1.png" width=700/>
<img src="../../data/images/fig3.png" width=500/>

By CrunchyRocks, after Karla Panchuck - https://openpress.usask.ca/physicalgeology/chapter/13-5-measuring-geological-structures/, CC BY 4.0, https://commons.wikimedia.org/w/index.php?curid=113554289

<a id='installing-gempy'></a>

# Installing GemPy

If you have not installed `GemPy` yet, please follow the [installation instructions](https://docs.gempy.org/installation.html). If you encounter any issues, feel free to open a new discussion at [GemPy Discussions](https://github.com/cgre-aachen/gempy/discussions). If you encounter an error in the installation process, feel free to also open an issue at [GemPy Issues](https://github.com/cgre-aachen/gempy/issues). There, the `GemPy` development team will help you out. 

<a id='importing-libraries'></a>

# Importing Libraries

For this notebook, we need the `pandas` library for the data preparation, `matplotlib` for plotting and of course the `gempy` library. The `gempy_viewer` library is used to plot the modeling results. Any warnings that may appear can be ignored for now. 

In [ ]:
import pandas as pd
import gempy as gp
import gempy_viewer as gpv
import matplotlib.pyplot as plt

Setting Backend To: AvailableBackends.numpy


<a id='data-preparation'></a>
# Data Preparation

For this model, the only thing that needs to be done is loading the already created interface points and orientations. In the next tutorials, you will create the data yourself and process it further to make it usable for GemPy. 

<a id='importing-interface-points'></a>
## Importing Interface Points

We are using the `pandas` library ([Pandas](https://pandas.pydata.org/docs/)) to load the interface points that were prepared beforehand and stored as CSV-file (comma-separated-file). The only information that is needed are the location of the interface point (`X`, `Y`, `Z`) and the `formation` it belongs to. You may have to adjust the `delimiter` (`'\t'`, `','`, `';'`, `' '`) when loading the file.

In [ ]:
interfaces = pd.read_csv('../../data/model1/model1_interfaces.csv', 
                         delimiter = '\t')
interfaces.head()

<a id='importing-orientations'></a>

## Importing Orientations

The orientations will also be loaded using `pandas`. In addition to the location and the formation the orientation belongs to, the dip value, azimuth value (dip direction) and a polarity value (mostly set to 1 by default) needs to be provided. As the model will feature horizontal layers, the dip is equal to 0. These three provided orientations belonging to `Layer1` are all the orientations that are needed to compute the model. There are no other orientations needed as the potential field approach implemented in `GemPy` allows to combine subparallel layers in on so-called `Series` where only at least one orientation is needed for the entire series.

In [ ]:
orientations = pd.read_csv('../../data/model1/model1_orientations.csv', 
                           delimiter='\t')
orientations.head()

<a id='gempy-model-calculation'></a>

# GemPy Model Calculation

The following part introduces the main steps of creating a model in `GemPy`. 

The creation of a `GemPy` Model follows particular steps which will be performed in the following:

1. Create new model and data initiation: `gp.create_geomodel()`
2. Map Stack to Surfaces: `gp.map_stack_to_surfaces()`
3. [...]
4. Computing the Model: `gp.compute_model()`

<a id='creating-the-gempy-model'></a>

## Creating the GemPy Model and Data Initiation

The first step is to create a new empty `GemPy` model by providing a name for it. 

In addition, the `extent` of the model (`xmin`, `xmax`, `ymin`, `ymax`, `zmin`, `zmax`) and the `resolution` in `X`, `Y`and `Z` direction (`res_x`, `res_y`, `res_z`, equal to the number of cells in each direction) will be set using lists of values. If you want to provide cells with a certain size, you would have to calculate the following. It is important to convert the resulting number of cells into an `int` as only integer values for the number of cells are valid. 

```python
res_x = int((xmax-xmin)/cell_size_x)
res_y = int((ymax-ymin)/cell_size_y)
res_z = int((zmax-zmin)/cell_size_z)
```

The interface points (`surface_points_df`) and orientations (`orientations_df`) will be passed as strings with the location of the CSV files. Please mind that you may have to edit the `pandas_reader_kwargs` by adding the `'sep'` key and the delimiter `'\t'` as item of a dictionary.

In [ ]:
geo_model: gp.data.GeoModel = gp.create_geomodel(
    project_name='Model1_Horizontal_Layers',
    extent=[0, 1000, 0, 1000, -600, 0], 
    resolution=[100, 100, 100],  # * Here we define the number of octree levels. If octree levels are defined, the resolution is ignored.
    importer_helper=gp.data.ImporterHelper(
        path_to_orientations='../../data/model1/model1_orientations.csv',
        path_to_surface_points='../../data/model1/model1_interfaces.csv',
        pandas_reader_kwargs = {'sep': '\t'}
    )
)

It is possible to check out the different attributes of the `GemPy` model object using `vars()`. It is mostly empty for now but will be filled in the following steps. The most important attributes/objects of the `GemPy` model are:

1. `meta`
2. `structural_frame`
3. `grid`
4. `geophysics_input`
5. `input_transform`
6. `interpolation_grid`


In [ ]:
vars(geo_model)

The attributes can easily be accessed via the `geo_model`.

In [ ]:
geo_model.meta

In [ ]:
geo_model.structural_frame

In [ ]:
geo_model.geophysics_input

In [ ]:
geo_model.input_transform

In [ ]:
geo_model.interpolation_grid

<a id='inspecting-the-surfaces'></a>

## Inspecting the Surfaces

The model consists of four different layers or surfaces now which all belong to the `Default series`. During the next step, the proper `Series` will be assigned to the surface. Using the `structural_frame`-attribute again, we can check which layers were loaded.

In [ ]:
geo_model.structural_frame

<a id='inspecting-the-input-data'></a>

## Inspecting the Input Data

The loaded interface points and orientations can again be inspected using the `surface_points`- and `orientations`-attributes. Using the `df`-attribute of this object will convert the displayed table in a `pandas` `DataFrame`.

In [ ]:
geo_model.surface_points_copy.df.head()

In [ ]:
geo_model.orientations_copy.df.head()

<a id='map-stack-to-surfaces'></a>

## Map Stack to Surfaces

We want our geological units to appear in the correct order relative to age. Such order might for example be given by a depositional sequence of stratigraphy, unconformities due to erosion or other lithological genesis events such as igneous intrusions. Defining the correct order of series is vital to the construction of the model!

During this step, all four layers of the model are assigned to the `Strata1` series. Per definition, only orientations for one layer are necessary to compute the model. However, it is of course possible to provide orientation measurements for the other layers as well. If the layers were not parallel as shown in the next models, multiple series would be defined. The order within one series also defines the age relations within this series and has to be according to the depositional events of the layers.

In [ ]:
gp.map_stack_to_surfaces(gempy_model=geo_model,
                         mapping_object=
                         {
                          'Strata1': ('Layer1', 'Layer2', 'Layer3', 'Layer4'),                          
                         },
                         remove_unused_series=True)
geo_model.structural_frame

<a id='plotting-the-input-data-in-2d'></a>

## Plotting the input data in 2D using Matplotlib

The input data can now be visualized in 2D using `matplotlib`. This might for example be useful to check if all points and measurements are defined the way we want them to. Using the function `plot_2d()`, we attain a 2D projection of our data points onto a plane of chosen direction (we can choose this attribute to be either `'x'`, `'y'`, or `'z'`).

In [ ]:
plot = gpv.plot_2d(geo_model, 
                   direction='z', 
                   show_lith=False, 
                   show_boundaries=False)

plt.grid()

<a id='plotting-the-input-data-in-3d'></a>

## Plotting the input data in 3D using PyVista

The input data can also be viszualized using the `pyvista` package. In this view, the interface points are visible as well as the orientations (marked as arrows) which indicate the normals of each orientation value. 

The `pyvista` package requires the Visualization Toolkit (VTK) to be installed.

In [ ]:
gpv.plot_3d(geo_model, 
            image=False, 
            show_lith=False,
            plotter_type='basic',
            kwargs_plotter= {'notebook':True},)

<a id='setting-the-interpolator'></a>
## Setting the interpolator

Unlike previous versions, GemPy 3 doesn’t rely on theano or asera. Instead, it utilizes numpy or tensorflow. Consequently, we no longer need to recompile all theano functions (TensorFlow uses eager execution; we found no notable speed difference after profiling the XLA compiler).

The parameters used for the interpolation are stored in gempy.core.data.GeoModel.interpolation_options. These parameters have sensible default values that you can modify if necessary. However, we advise caution when changing these parameters unless you fully understand their implications.

Display the current interpolation options

In [ ]:
geo_model.interpolation_options

<a id='computing-the-model'></a>

## Computing the model

At this point, we have all we need to compute our full model via `gp.compute_model()`. By default, this will return two separate solutions in the form of arrays. The first provides information on the lithological formations, the second on the fault network in the model, which is not present in this example. 

In [ ]:
sol = gp.compute_model(geo_model, 
                       compute_mesh=True)

In [ ]:
sol

In [ ]:
geo_model.solutions

<a id='model-visualization-and-post-processing'></a>

# Model Visualization and Post-Processing

<a id='visualizing-cross-sections-of-the-computed-model'></a>

## Visulazing Cross Sections of the computed model

Cross sections in different `direction`s and at different `cell_number`s can be displayed. Here, we only see the horizontal layers of the model. Not very much exciting yet. 

In [ ]:
gpv.plot_2d(geo_model, 
           direction=['x', 'x', 'y', 'y'], 
           cell_number=[25, 50, 25, 75], 
           show_topography=False, 
           show_data=True)

Next to the lithology data, we can also plot the calculated scalar field.

In [ ]:
gpv.plot_2d(geo_model, show_data=False, show_scalar=True, show_lith=False)

<a id='visualizing-the-computed-model-in-3d'></a>

## Visualizing the computed model in 3D

The computed model can be visualized in 3D using the `pyvista` library. Setting `notebook=False` will open an interactive windows and the model can be rotated and zooming is possible. 

In [ ]:
gpv.plot_3d(geo_model, 
                 image=False, 
                 show_topography=True,
                 plotter_type='basic', 
                 kwargs_plotter= {'notebook':True},
                 show_lith=True)

<a id='conclusions'></a>
# Conclusions

<div class="alert alert-block alert-success">
<b>In this tutorial, you learnt the following:</b> <br>
- How to import input data into <b>GemPy</b> via CSV-Files (comma-separated-values) and what the files have to look like<br>
- How to build a simple model consisting of <b>horizontal layers</b> belonging to <b>one Series</b><br>
- How to visualize the resulting model with cross sections in 2D and the entire model in 3D

</div>


<a id='outlook'></a>
# Outlook

<div class="alert alert-block alert-success">
<b>In the next tutorial, you will learn the following:</b> <br>
- Get a better undestanding of what orientations mean in GemPy and how to plot them using <b>mplstereonet</b><br>
- How to build a simple model consisting of <b>folded layers</b> belonging to <b>one Series</b><br>


</div>

[Take me to the next notebook on Github](https://nbviewer.org/github/cgre-aachen/gemgis_data/blob/main/notebooks/01_basic_modeling/model2_Folded_Layers.ipynb)

[Take me to the next notebook locally](model2_Folded_Layers.ipynb)

<img src="../../jose/images/fig1.png" />

<a id='licensing'></a>

## Licensing

Institute for Computational Geoscience, Geothermics and Reservoir Geophysics, RWTH Aachen University & Fraunhofer IEG, Fraunhofer Research Institution for Energy Infrastructures and Geotechnologies IEG, Authors: Alexander Juestel. For more information contact: alexander.juestel(at)ieg.fraunhofer.de

All notebooks are licensed under a Creative Commons Attribution 4.0 International License (CC BY 4.0, http://creativecommons.org/licenses/by/4.0/). References for each displayed map are provided. Most of the maps originate from the books of [Powell (1992)](https://link.springer.com/book/9783540586074) and [Bennison (1990)](https://link.springer.com/book/10.1007/978-1-4615-9630-1). References for maps with unknown origin will gladly be added.